In [2]:
library("INLA")
library("dplyr")
library("tidyverse")

In [3]:
data = read.csv("JHU_rollup_as_of_2020-10-01.csv", header = TRUE)
county = read.csv("county_pop.csv",header = TRUE)
data = merge(data,county, by="fips")

lag.data = data %>% group_by(fips) %>%arrange(date)%>% mutate(daily = confirmed - lag(confirmed, 1))

lag.data %>% mutate(daily2 = ifelse(is.na(daily),0, daily)) ->lag.noNA #daily2 NA values replaced with 0 

lag.noNA %>% mutate(daily2 = ifelse(daily < 0, 0, daily)) ->lag.clean  #negative daily2 values replaced with 0

#see how many lags are negative
pos<-nrow(lag.data[lag.data$daily>=0,])
neg<-nrow(lag.data[lag.data$daily<0,])
percent.neg<- neg/(neg+pos) #Note: Only 2% of the daily observations are negative. We will replace these with zero

In [4]:
# Create a temporal model with no spatial term
sp.observation = lag.clean %>% group_by(fips)%>% count(fips)
sp = 2826 #this is the number of different counties

lag.clean%>% distinct() %>% group_by(fips, date) %>% filter(deaths==max(deaths))%>% distinct() %>% filter(daily2 ==max(daily2))-> data.maxdeath

num_observation = data.maxdeath%>%group_by(fips)%>%count(fips)

data.maxdeath%>%group_by(fips, date)%>%filter(confirmed==max(confirmed))->data.maxconfirmed
data.maxconfirmed%>%group_by(fips, date)%>%filter(daily==max(daily))->data.maxdaily
num_observation2= data.maxdaily%>%group_by(fips)%>%count(fips)

#THE DATA IS ALL CLEAN! No NAs or negatives for daily2, 192 observations for each fip
JHU.data<-data.maxdaily

In [5]:
# Make an ID.day variable. Goes from 1 to n and identifies the counties
JHU.data %>% group_by(fips) %>% arrange(date)%>% 
   mutate(id.Day = rep(1:192, times=as.factor(sp)))->ID.data

ID.data %>% mutate(logPop = rep(log(population)))-> pop.data #this is base e, not 10

subset.data <- pop.data%>% filter(1 < id.Day & id.Day<150)

In [ ]:
#should it be model = ar1 instead of rw1
formula.temp2 <- daily2 ~ 1 + f(id.Day, model="ar1")
model.temp2<-inla(formula.temp2, family="poisson",data=subset.data,offset=logPop,verbose = TRUE, control.predictor = list(compute=TRUE))
summary(model.temp2)

In [ ]:
plot(model.temp2, plot.prior = TRUE) #Th is should plot prior and posterior on the same plot

In [ ]:
fitted <- model.temp2$summary.fitted.values #df with mean, stdev, and quantiles of the fitted values
predicted <- model.temp2$summary.linear.predictor #df with mean, stdev, and quantiles of the linear predictors
marginals.pred <- model.temp2$marginals.linear.predictor #computes a list with the posterior marginals of the linear predictors
marginals.fit <- model.temp2$marginals.fitted.values # list with a the posterior marginals of the fitted values

In [ ]:
plot(fitted$mean, predicted$mean) #probably not a good fit; all over the place. Predictions are way smaller than the actual values. Some other things going on the model doesn't capture. Predictions up to 200 while actual 6000.
#these large values are NYC
# The y=x line is where you are fitting well. All the values below are where you are underpredicting
# We also have to account for the zeros
# Temporal only model